In [1]:
import torch
import math

import numpy as np
import torch.nn as nn
import torch.nn.functional as F


from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
BATCH_SIZE=100

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        ])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
eval_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
eval_loader = torch.utils.data.DataLoader(dataset=eval_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [4]:
class IAF_Layer(nn.Module):
    def __init__(self,  input_dim, hidden_dim, out_dim, reverse=False):
        super(IAF_Layer, self).__init__()
        
        self.m_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim, out_dim)
            )
        self.s_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim, out_dim)
            )
        
        self.sigmoid = nn.Sigmoid()
        
        self.reverse = reverse
    
    def forward(self, z, h):
        
#         print(z.shape)
#         exit()
#         if self.reverse:
#             z =  torch.flip(z, dims=-1)
        
        iaf_input = torch.cat([z, h], dim=-1)
        m = self.m_layer(iaf_input)
        s = self.s_layer(iaf_input)
        sigma = self.sigmoid(s)
    
        return z * sigma + m * (1. - sigma), sigma

In [5]:
class IAF_VAE(nn.Module):
    def __init__(self, x_dim, h1, h2, z_dim, num_iaf):
        super(IAF_VAE, self).__init__()
        
        self.enc = nn.Sequential(
            nn.Linear(x_dim, h1),
            nn.ReLU(),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Linear(h2, z_dim),
            nn.ReLU()
        )
        
        self.enc_mu = nn.Linear(z_dim, z_dim)
        self.enc_log_var = nn.Linear(z_dim, z_dim)
        
        iaf_layers = []
        for i in range(num_iaf):
            reverse = i > 0 
            iaf_layers.append(IAF_Layer(z_dim*2, z_dim*2, z_dim, reverse=reverse))
        self.iaf_layers = nn.ModuleList(iaf_layers)
        
        self.dec = nn.Sequential(
            nn.Linear(z_dim, h2),
            nn.ReLU(),
            nn.Linear(h2, h1),
            nn.ReLU(),
            nn.Linear(h1, x_dim),
            nn.Sigmoid()
            )
    
    def forward(self, x):
        # original z_0
        h = self.enc(x)
        mu, log_var = self.enc_mu(h), self.enc_log_var(h)
        z_0, eps_0 = self.sampling(mu, log_var)
        
        z_t = z_0
        z_sigmas = []
        
        # iaf flows 
        for iaf in self.iaf_layers:
            z_t, sigma_z_t = iaf(z_t, h)
            z_sigmas.append(sigma_z_t)
        
        z = z_t 
        
        return self.dec(z), mu, log_var, z_sigmas, eps_0, z
        
    def sampling(self, mu, log_var):
        # reparametrization trick
        std = torch.exp(0.5*log_var)
        eps = torch.rand_like(std)
        return mu + (eps * std), eps

vae = IAF_VAE(x_dim=784, h1=512, h2=256, z_dim=2, num_iaf=4)

In [6]:
input_sample = torch.randn(10, 784)

In [7]:
res, mu, log_var, z_sigmas, eps_0, z_t = vae(input_sample)

In [8]:
def loss_fn(recon_x, x, mu, log_var, z_sigmas, eps_0, z):
    # reconstruction loss : binary cross entropy 
    # logp(x|z)
    bce_loss = F.binary_cross_entropy(recon_x, x, reduction='sum')
    
    # -logp(z)
    logpz = torch.sum(0.5 * np.log(2*math.pi) + 0.5*z**2)
    
    # logq(z|x)
    det = log_var 
    for z_sigma in z_sigmas:
        det += torch.log(z_sigma)
    logqz_x = torch.sum(0.5 * np.log(2*math.pi) +0.5*eps_0**2 + det)
    
    return bce_loss + logpz - logqz_x

In [9]:
optimizer = torch.optim.Adam(vae.parameters())

In [10]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_ind, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.view(BATCH_SIZE, -1)
        recon_x, mu, log_var, z_sigmas, eps_0, z_t = vae(data)
        loss = loss_fn(recon_x, data, mu, log_var, z_sigmas, eps_0, z_t)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_ind % 200 == 0:
            print('Train Epoch:{} [{}/{} ({:0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_ind*len(data), len(train_loader.dataset),
                100*batch_ind/len(train_loader), loss.item()/len(data)))
        
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss/len(train_loader.dataset)))

In [11]:
def evaluation(): 
    vae.eval()
    eval_loss = 0
    with torch.no_grad():
        for data, _ in eval_loader:
#             data = data.cuda()
            data = data.view(BATCH_SIZE, -1)
            recon_x, mu, log_var, z_sigmas, eps_0, z_t = vae(data)
            eval_loss += loss_fn(recon_x, data, mu, log_var, z_sigmas, eps_0, z_t)
            
    eval_loss /= len(eval_loader.dataset)
    print('====> Evaluation loss : {:.4f}'.format(eval_loss))   
    return eval_loss

In [12]:
best_loss = float('inf')
for epoch in range(1, 200):
    train(epoch)
    eval_loss = evaluation()
    if eval_loss < best_loss:
        torch.save(vae.state_dict(), 'iaf_vae_best.pth')
        best_loss = eval_loss

Train Epoch:1 [0/60000 (0.000000%)]	Loss: 549.676680
Train Epoch:1 [20000/60000 (33.333333%)]	Loss: 188.982480
Train Epoch:1 [40000/60000 (66.666667%)]	Loss: 185.670273
====> Epoch: 1 Average loss: 191.6321
====> Evaluation loss : 177.9373
Train Epoch:2 [0/60000 (0.000000%)]	Loss: 180.321699
Train Epoch:2 [20000/60000 (33.333333%)]	Loss: 178.761211
Train Epoch:2 [40000/60000 (66.666667%)]	Loss: 168.771289
====> Epoch: 2 Average loss: 173.9770
====> Evaluation loss : 169.2890
Train Epoch:3 [0/60000 (0.000000%)]	Loss: 170.882656
Train Epoch:3 [20000/60000 (33.333333%)]	Loss: 171.793438
Train Epoch:3 [40000/60000 (66.666667%)]	Loss: 167.121563
====> Epoch: 3 Average loss: 167.3482
====> Evaluation loss : 163.7703
Train Epoch:4 [0/60000 (0.000000%)]	Loss: 164.443945
Train Epoch:4 [20000/60000 (33.333333%)]	Loss: 155.803955
Train Epoch:4 [40000/60000 (66.666667%)]	Loss: 152.185020
====> Epoch: 4 Average loss: 162.1096
====> Evaluation loss : 160.1425
Train Epoch:5 [0/60000 (0.000000%)]	Loss

====> Evaluation loss : 148.5002
Train Epoch:35 [0/60000 (0.000000%)]	Loss: 143.940605
Train Epoch:35 [20000/60000 (33.333333%)]	Loss: 145.012432
Train Epoch:35 [40000/60000 (66.666667%)]	Loss: 144.996094
====> Epoch: 35 Average loss: 149.3624
====> Evaluation loss : 148.3037
Train Epoch:36 [0/60000 (0.000000%)]	Loss: 146.740137
Train Epoch:36 [20000/60000 (33.333333%)]	Loss: 169.808418
Train Epoch:36 [40000/60000 (66.666667%)]	Loss: 143.752813
====> Epoch: 36 Average loss: 151.4378
====> Evaluation loss : 148.5747
Train Epoch:37 [0/60000 (0.000000%)]	Loss: 151.506494
Train Epoch:37 [20000/60000 (33.333333%)]	Loss: 140.557920
Train Epoch:37 [40000/60000 (66.666667%)]	Loss: 142.052051
====> Epoch: 37 Average loss: 147.6577
====> Evaluation loss : 145.9325
Train Epoch:38 [0/60000 (0.000000%)]	Loss: 159.409443
Train Epoch:38 [20000/60000 (33.333333%)]	Loss: 153.887002
Train Epoch:38 [40000/60000 (66.666667%)]	Loss: 150.508398
====> Epoch: 38 Average loss: 150.8191
====> Evaluation loss : 

Train Epoch:68 [40000/60000 (66.666667%)]	Loss: 144.755937
====> Epoch: 68 Average loss: 157.3677
====> Evaluation loss : 149.9432
Train Epoch:69 [0/60000 (0.000000%)]	Loss: 155.387002
Train Epoch:69 [20000/60000 (33.333333%)]	Loss: 143.343203
Train Epoch:69 [40000/60000 (66.666667%)]	Loss: 141.501309
====> Epoch: 69 Average loss: 148.5510
====> Evaluation loss : 147.2831
Train Epoch:70 [0/60000 (0.000000%)]	Loss: 141.326768
Train Epoch:70 [20000/60000 (33.333333%)]	Loss: 154.369043
Train Epoch:70 [40000/60000 (66.666667%)]	Loss: 168.941641
====> Epoch: 70 Average loss: 155.0468
====> Evaluation loss : 154.4161
Train Epoch:71 [0/60000 (0.000000%)]	Loss: 158.854766
Train Epoch:71 [20000/60000 (33.333333%)]	Loss: 146.564063
Train Epoch:71 [40000/60000 (66.666667%)]	Loss: 152.551338
====> Epoch: 71 Average loss: 148.4194
====> Evaluation loss : 143.3674
Train Epoch:72 [0/60000 (0.000000%)]	Loss: 151.759180
Train Epoch:72 [20000/60000 (33.333333%)]	Loss: 142.615703
Train Epoch:72 [40000/60

Train Epoch:102 [20000/60000 (33.333333%)]	Loss: 135.744795
Train Epoch:102 [40000/60000 (66.666667%)]	Loss: 153.698457
====> Epoch: 102 Average loss: 139.0070
====> Evaluation loss : 135.0565
Train Epoch:103 [0/60000 (0.000000%)]	Loss: 135.543125
Train Epoch:103 [20000/60000 (33.333333%)]	Loss: 139.440156
Train Epoch:103 [40000/60000 (66.666667%)]	Loss: 132.971816
====> Epoch: 103 Average loss: 135.5384
====> Evaluation loss : 134.5070
Train Epoch:104 [0/60000 (0.000000%)]	Loss: 145.522207
Train Epoch:104 [20000/60000 (33.333333%)]	Loss: 150.885996
Train Epoch:104 [40000/60000 (66.666667%)]	Loss: 129.267988
====> Epoch: 104 Average loss: 136.4206
====> Evaluation loss : 132.9056
Train Epoch:105 [0/60000 (0.000000%)]	Loss: 130.750059
Train Epoch:105 [20000/60000 (33.333333%)]	Loss: 130.874580
Train Epoch:105 [40000/60000 (66.666667%)]	Loss: 144.994668
====> Epoch: 105 Average loss: 136.4821
====> Evaluation loss : 133.3910
Train Epoch:106 [0/60000 (0.000000%)]	Loss: 130.871016
Train Ep

Train Epoch:135 [40000/60000 (66.666667%)]	Loss: 128.710010
====> Epoch: 135 Average loss: 122.9564
====> Evaluation loss : 122.8700
Train Epoch:136 [0/60000 (0.000000%)]	Loss: 129.655801
Train Epoch:136 [20000/60000 (33.333333%)]	Loss: 116.325977
Train Epoch:136 [40000/60000 (66.666667%)]	Loss: 121.728340
====> Epoch: 136 Average loss: 122.1296
====> Evaluation loss : 123.2038
Train Epoch:137 [0/60000 (0.000000%)]	Loss: 117.189365
Train Epoch:137 [20000/60000 (33.333333%)]	Loss: 114.849961
Train Epoch:137 [40000/60000 (66.666667%)]	Loss: 122.137793
====> Epoch: 137 Average loss: 121.3809
====> Evaluation loss : 119.6129
Train Epoch:138 [0/60000 (0.000000%)]	Loss: 121.444297
Train Epoch:138 [20000/60000 (33.333333%)]	Loss: 121.890332
Train Epoch:138 [40000/60000 (66.666667%)]	Loss: 126.097266
====> Epoch: 138 Average loss: 120.5838
====> Evaluation loss : 118.5706
Train Epoch:139 [0/60000 (0.000000%)]	Loss: 115.100938
Train Epoch:139 [20000/60000 (33.333333%)]	Loss: 123.882295
Train Ep

====> Epoch: 168 Average loss: 85.1402
====> Evaluation loss : 81.8840
Train Epoch:169 [0/60000 (0.000000%)]	Loss: 88.184619
Train Epoch:169 [20000/60000 (33.333333%)]	Loss: 87.017490
Train Epoch:169 [40000/60000 (66.666667%)]	Loss: 72.539395
====> Epoch: 169 Average loss: 83.1685
====> Evaluation loss : 80.6585
Train Epoch:170 [0/60000 (0.000000%)]	Loss: 82.178691
Train Epoch:170 [20000/60000 (33.333333%)]	Loss: 79.987148
Train Epoch:170 [40000/60000 (66.666667%)]	Loss: 88.772979
====> Epoch: 170 Average loss: 81.8494
====> Evaluation loss : 79.3610
Train Epoch:171 [0/60000 (0.000000%)]	Loss: 74.618408
Train Epoch:171 [20000/60000 (33.333333%)]	Loss: 87.100293
Train Epoch:171 [40000/60000 (66.666667%)]	Loss: 72.497363
====> Epoch: 171 Average loss: 80.5863
====> Evaluation loss : 77.9117
Train Epoch:172 [0/60000 (0.000000%)]	Loss: 71.051660
Train Epoch:172 [20000/60000 (33.333333%)]	Loss: 71.968486
Train Epoch:172 [40000/60000 (66.666667%)]	Loss: 83.105977
====> Epoch: 172 Average los

RuntimeError: all elements of input should be between 0 and 1

In [13]:
vae.load_state_dict(torch.load('iaf_vae_best.pth'))
_ = vae.eval()

In [14]:
with torch.no_grad():
    for batch_ind, (data, _) in enumerate(eval_loader):
#             data = data.cuda()
        
        data = data.view(BATCH_SIZE, -1)
        recon_batch, *_ = vae(data)
        n = min(data.size(0), 8)
        comparison = torch.cat([data[:n].view(8, 1, 28, 28)[:n],
                                    recon_batch.unsqueeze(-1).view(BATCH_SIZE, 1, 28, 28)[:n]])
        save_image(comparison.data.cpu(),
                       'samples/sample_comp_iaf' +'.png', nrow=n)
        break

In [15]:
with torch.no_grad():
    z = torch.randn(64, 2)
    sample = vae.dec(z).view(64, 1, 28, 28)
    save_image(sample.data.cpu(), 'samples/sample_iaf.png', nrow=16)